In [1]:
from keras.models import Model
from keras.layers import Input, Bidirectional, Concatenate, Dot
from keras.layers.core import Activation, Dense, Lambda
from keras.layers.convolutional import Conv1D, Cropping1D, ZeroPadding1D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import add, multiply
from keras import regularizers, initializers
import keras.backend as kb
from keras.losses import binary_crossentropy, mean_squared_error, categorical_crossentropy
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import numpy as np


Using TensorFlow backend.


In [2]:
# input_org = Input(shape=(51, 4), name='input_org')
# input_rev = Input(shape=(51, 4), name='input_rev')

# conv1 = Conv1D(filters=8, kernel_size=1, padding='same', activation=None)(input_org)
# conv1_rv = Conv1D(filters=8, kernel_size=1, padding='same', activation=None)(input_rev)
# conv1_rv_mid = Cropping1D(25)(conv1_rv)

# def repeat_vector(args):
#     layer_input = args[0]
#     # hard-coded for now
#     return kb.repeat_elements(kb.expand_dims(layer_input, 0), rep=51, axis=0)

# conv1_rv_mid_tile = Lambda(repeat_vector)(conv1_rv_mid)

# # def element_wise_multiply(x):
# #     image,mask = x
# #     mask = kb.expand_dims(mask, axis=-1) #could be K.stack([mask]*3, axis=-1) too 
# #     return mask*image

# # conv1_prod = element_wise_multiply([conv1, conv1_rv_mid_tile])
# conv1_prod = multiply([conv1, conv1_rv_mid_tile])

# model = Model(input=[input_org, input_rev], output=conv1_prod)

# model.compile(loss='binary_crossentropy', optimizer='adam')

In [3]:
input_org = Input(shape=(51, 4), name='input_org')
input_rev = Input(shape=(51, 4), name='input_rev')

conv1 = Conv1D(filters=8, kernel_size=1, padding='same', activation=None)(input_org)
conv1_rv = Conv1D(filters=8, kernel_size=1, padding='same', activation=None)(input_rev)
conv1_rv_mid = Cropping1D(25)(conv1_rv)

def swap_axes(args):
    layer_input = args[0]
    # hard-coded for now
    return kb.permute_dimensions(layer_input, (1, 0))

# conv1_rv_mid_swap = Lambda(swap_axes)(conv1_rv_mid)

# def element_wise_multiply(x):
#     image,mask = x
#     mask = kb.expand_dims(mask, axis=-1) #could be K.stack([mask]*3, axis=-1) too 
#     return mask*image

# conv1_prod = element_wise_multiply([conv1, conv1_rv_mid_tile])
conv1_prod = Dot(axes=-1)([conv1, conv1_rv_mid])

model = Model(input=[input_org, input_rev], output=conv1_prod)

model.compile(loss='binary_crossentropy', optimizer='adam')

/Users/alicegao/anaconda2/envs/dg_work/lib/python2.7/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("do..., inputs=[<tf.Tenso...)`


In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_rev (InputLayer)          (None, 51, 4)        0                                            
__________________________________________________________________________________________________
input_org (InputLayer)          (None, 51, 4)        0                                            
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 51, 8)        40          input_rev[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 51, 8)        40          input_org[0][0]                  
__________________________________________________________________________________________________
cropping1d